In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [5]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("/Users/madahbar/Documents/IRONHACK/Week 4/LABS/lab-natural-language-processing/data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)
data.head()


(1000, 2)


,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


### Let's divide the training and test set into two partitions


In [6]:
from sklearn.model_selection import train_test_split

# Assume your dataframe is called 'data'
# and you have a 'label' column that is 0 (ham) or 1 (spam)
X = data['text']          # features
y = data['label']         # targets

# Split into train and validation sets (80%/20%)
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,        # 20% for validation
    random_state=42,      # reproducibility
    stratify=y            # keep same label distribution
)

print("Train size:", X_train.shape[0])
print("Validation size:", X_val.shape[0])


Train size: 800
Validation size: 200


## Data Preprocessing

## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [7]:
from bs4 import BeautifulSoup
import re

def clean_html(text):
    # Remove JavaScript/CSS sections
    text = re.sub(r'(<script.*?>.*?</script>)|(<style.*?>.*?</style>)', '', text, flags=re.DOTALL)
    
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text)
    
    # Remove remaining HTML tags
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()
    
    return text


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [8]:
def clean_text(text):
    # Remove special characters
    text = re.sub(r'\W', ' ', text)
    
    # Remove all single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    
    # Remove single characters from start
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Substitute multiple spaces with single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    
    # Remove prefixed 'b' (common from bytes decoding)
    text = re.sub(r'^b\s+', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    return text


## Now let's work on removing stopwords
Remove the stopwords.

In [9]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    tokens = text.split()
    filtered_words = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_words)


In [10]:
# Apply full cleaning pipeline
data['clean_text'] = data['text'].apply(lambda x: clean_html(x))
data['clean_text'] = data['clean_text'].apply(lambda x: clean_text(x))
data['clean_text'] = data['clean_text'].apply(lambda x: remove_stopwords(x))

# Show a preview
data[['text', 'clean_text']].head()

/var/folders/3b/sqzcq4m91w14zr5xb30wbqnm0000gn/T/ipykernel_31560/2694515195.py:12: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  soup = BeautifulSoup(text, "html.parser")


,text,clean_text
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",dear sir strictly private business proposal mi...
1,Will do.,
2,Nora--Cheryl has emailed dozens of memos about...,nora cheryl emailed dozens memos haiti weekend...
3,Dear Sir=2FMadam=2C I know that this proposal ...,dear sir fmadam c know proposal might surprise...
4,fyi,fyi


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [12]:
# If your cleaned text column is called 'clean_text', alias it to match the lab:
if 'preprocessed_text' not in data.columns and 'clean_text' in data.columns:
    data['preprocessed_text'] = data['clean_text']

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag

# One-time downloads (safe to run multiple times)
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

def _to_wn_tag(treebank_tag):
    if treebank_tag.startswith('J'): return wordnet.ADJ
    if treebank_tag.startswith('V'): return wordnet.VERB
    if treebank_tag.startswith('N'): return wordnet.NOUN
    if treebank_tag.startswith('R'): return wordnet.ADV
    return wordnet.NOUN  # default

def lemmatize_text(s: str) -> str:
    tokens = s.split()
    tagged = pos_tag(tokens)
    lemm = [lemmatizer.lemmatize(tok, _to_wn_tag(tag)) for tok, tag in tagged]
    return " ".join(lemm)

# Apply to the whole training dataframe (fast enough for this lab)
data['preprocessed_text'] = data['preprocessed_text'].astype(str).apply(lemmatize_text)
data[['text','preprocessed_text']].head()


[nltk_data] Downloading package punkt to /Users/madahbar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/madahbar/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/madahbar/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/madahbar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/madahbar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,text,preprocessed_text
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",dear sir strictly private business proposal mi...
1,Will do.,
2,Nora--Cheryl has emailed dozens of memos about...,nora cheryl email dozen memo haiti weekend ple...
3,Dear Sir=2FMadam=2C I know that this proposal ...,dear sir fmadam c know proposal might surprise...
4,fyi,fyi


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [13]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Separate by class
ham_texts  = data.loc[data['label'] == 1, 'preprocessed_text']  # adjust if your lab uses different 0/1 meaning
spam_texts = data.loc[data['label'] == 0, 'preprocessed_text']

# Vectorize (no stopwords now because we already removed them earlier)
cv = CountVectorizer()
ham_mat  = cv.fit_transform(ham_texts)
ham_vocab = cv.get_feature_names_out()
ham_counts = pd.Series(ham_mat.sum(axis=0).A1, index=ham_vocab).sort_values(ascending=False).head(10)

# Refit a fresh vectorizer for spam (keeps each class independent for top terms)
cv_spam = CountVectorizer()
spam_mat  = cv_spam.fit_transform(spam_texts)
spam_vocab = cv_spam.get_feature_names_out()
spam_counts = pd.Series(spam_mat.sum(axis=0).A1, index=spam_vocab).sort_values(ascending=False).head(10)

print("Top 10 HAM words\n", ham_counts, "\n")
print("Top 10 SPAM words\n", spam_counts)


Top 10 HAM words
 money          981
account        907
bank           825
fund           787
transfer       568
transaction    551
business       514
country        508
mr             456
million        452
dtype: int64 

Top 10 SPAM words
 state        136
pm           127
call         115
say          111
work         108
would        107
president     98
time          96
mr            90
get           87
dtype: int64


## Extra features

In [15]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

# Create training dataframe with preprocessed text
data_train = pd.DataFrame({
	'preprocessed_text': X_train,
	'label': y_train
})

# Create validation dataframe with preprocessed text  
data_val = pd.DataFrame({
	'preprocessed_text': X_val,
	'label': y_val
})

# Add extra features to training data
data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list, case=False, na=False)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words, case=False, na=False)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(str(x))) 

# Add extra features to validation data
data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list, case=False, na=False)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words, case=False, na=False)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(str(x))) 

data_train.head()

,preprocessed_text,label,money_mark,suspicious_words,text_len
442,Dear=2C Good day hope fine=2Cdear am writting ...,1,1,1,1609
962,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...,1,1,1,3123
971,Will do.,0,0,0,8
190,FROM THE DESK OF DR.ADAMU ISMALERAUDITING AND...,1,1,1,530
551,"Dear Friend, My name is LOI C.ESTRADA,The wife...",1,1,1,2126


## How would work the Bag of Words with Count Vectorizer concept?

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

# Use the preprocessed text
X_text = data['preprocessed_text'].astype(str)

# Fit on the whole dataset (or only on train if you already split)
cv = CountVectorizer(ngram_range=(1,2), min_df=2)   # unigrams+bigrams, drop ultra-rare terms
X_bow = cv.fit_transform(X_text)

print("Vocabulary size:", len(cv.vocabulary_))
print("BoW matrix shape:", X_bow.shape)   # (n_samples, n_features)


Vocabulary size: 16792
BoW matrix shape: (1000, 16792)


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=2, sublinear_tf=True)
X_tfidf = tfidf.fit_transform(data['preprocessed_text'].astype(str))

print("Vocabulary size:", len(tfidf.vocabulary_))
print("TF-IDF matrix shape:", X_tfidf.shape)


Vocabulary size: 16792
TF-IDF matrix shape: (1000, 16792)


## And the Train a Classifier?

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

X = data['preprocessed_text'].astype(str)
y = data['label']

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Baseline 1: BoW + MultinomialNB
bow_nb = Pipeline([
    ("cv", CountVectorizer(ngram_range=(1,2), min_df=2)),
    ("clf", MultinomialNB())  # default params per lab
])
bow_nb.fit(X_train, y_train)
pred_nb = bow_nb.predict(X_val)
print("BoW + NB accuracy:", accuracy_score(y_val, pred_nb))
print(classification_report(y_val, pred_nb))

# Baseline 2: TF-IDF + Logistic Regression (optional, strong linear baseline)
tfidf_lr = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=2, sublinear_tf=True)),
    ("clf", LogisticRegression(max_iter=1000))
])
tfidf_lr.fit(X_train, y_train)
pred_lr = tfidf_lr.predict(X_val)
print("TF-IDF + LR accuracy:", accuracy_score(y_val, pred_lr))


BoW + NB accuracy: 0.975
              precision    recall  f1-score   support

           0       0.99      0.96      0.98       112
           1       0.96      0.99      0.97        88

    accuracy                           0.97       200
   macro avg       0.97      0.98      0.97       200
weighted avg       0.98      0.97      0.98       200

TF-IDF + LR accuracy: 0.985


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [19]:
import numpy as np
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler

# Ensure extra features exist
if not {'money_mark','suspicious_words','text_len'}.issubset(data.columns):
    # quick creation (same as earlier cell)
    money_simbol_list = r"|".join(["euro","dollar","pound","€","£","$"])
    suspicious_words = r"|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])
    data['money_mark']      = data['preprocessed_text'].str.contains(money_simbol_list, regex=True).astype(int)
    data['suspicious_words']= data['preprocessed_text'].str.contains(suspicious_words, regex=True).astype(int)
    data['text_len']        = data['preprocessed_text'].apply(len)

# Split once so all experiments are comparable
X_train_txt, X_val_txt, y_train, y_val = train_test_split(
    data['preprocessed_text'].astype(str), data['label'],
    test_size=0.2, random_state=42, stratify=data['label']
)

# ----- (A) BoW only -----
cv = CountVectorizer(ngram_range=(1,2), min_df=2)
Xtr_bow = cv.fit_transform(X_train_txt)
Xva_bow = cv.transform(X_val_txt)

clf_nb = MultinomialNB()
clf_nb.fit(Xtr_bow, y_train)
pred_A = clf_nb.predict(Xva_bow)

# ----- (B) TF-IDF only -----
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=2, sublinear_tf=True)
Xtr_tfidf = tfidf.fit_transform(X_train_txt)
Xva_tfidf = tfidf.transform(X_val_txt)

clf_nb_B = MultinomialNB()
clf_nb_B.fit(Xtr_tfidf, y_train)
pred_B = clf_nb_B.predict(Xva_tfidf)

# ----- (C) TF-IDF + extra flags -----
# Build extra features matrices aligned to the same splits
extra_train = data.loc[X_train_txt.index, ['money_mark','suspicious_words','text_len']].copy()
extra_val   = data.loc[X_val_txt.index,   ['money_mark','suspicious_words','text_len']].copy()

# Scale numeric extras (NB expects non-negative; StandardScaler(with_mean=False) keeps sparsity safe)
scaler = StandardScaler(with_mean=False)
extra_train_scaled = scaler.fit_transform(extra_train)
extra_val_scaled   = scaler.transform(extra_val)

Xtr_combo = hstack([Xtr_tfidf, extra_train_scaled])
Xva_combo = hstack([Xva_tfidf, extra_val_scaled])

clf_nb_C = MultinomialNB()
clf_nb_C.fit(Xtr_combo, y_train)
pred_C = clf_nb_C.predict(Xva_combo)

# -------- Results ----------
from sklearn.metrics import accuracy_score
print(f"(A) BoW + NB:       {accuracy_score(y_val, pred_A):.4f}")
print(f"(B) TF-IDF + NB:    {accuracy_score(y_val, pred_B):.4f}")
print(f"(C) TF-IDF + flags: {accuracy_score(y_val, pred_C):.4f}")


(A) BoW + NB:       0.9750
(B) TF-IDF + NB:    0.9750
(C) TF-IDF + flags: 0.9750
